In [1]:
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y


tools = [multiply, exponentiate, add]

gpt35 = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0).bind_tools(tools)
claude3 = ChatAnthropic(model="claude-3-sonnet-20240229").bind_tools(tools)
llm_with_tools = gpt35.configurable_alternatives(
    ConfigurableField(id="llm"), default_key="gpt35", claude3=claude3
)

/Users/chestercurme/repos/langchain/libs/core/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


# AgentExecutor

In [2]:
from typing import List, Tuple, Union

from langchain.agents import AgentExecutor
from langchain.agents.output_parsers.openai_tools import OpenAIToolAgentAction
from langchain_core.agents import AgentFinish, _convert_agent_action_to_messages
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


def actions_observations_to_messages(
    steps: List[Tuple[OpenAIToolAgentAction, str]],
) -> List[BaseMessage]:
    messages = []
    for action, observation in steps:
        messages.extend([m for m in action.message_log if m not in messages])
        messages.append(ToolMessage(observation, tool_call_id=action.tool_call_id))
    return messages


def messages_to_action(
    msg: AIMessage,
) -> Union[List[OpenAIToolAgentAction], AgentFinish]:
    if isinstance(msg, AIMessage) and msg.tool_calls is not None:
        actions = []
        for tool_call in msg.tool_calls:
            actions.append(
                OpenAIToolAgentAction(
                    tool=tool_call.name,
                    tool_input=tool_call.args,
                    tool_call_id=tool_call.id,
                    message_log=[msg],
                    log="",
                )
            )
        return actions
    else:
        return AgentFinish(return_values={"output": msg.content}, log="")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful assistant with access to tools"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: actions_observations_to_messages(
            x["intermediate_steps"]
        ),
    )
    | prompt
    | llm_with_tools
    | messages_to_action
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [3]:
agent_executor.invoke(
    {"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"}
)



> Entering new AgentExecutor chain...
300.03770462067547-900.8841

> Finished chain.


{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': 'The result of \\(3 + 5^{2.743}\\) is approximately 300.04, and the result of \\(17.24 - 918.1241\\) is approximately -900.88.'}

In [4]:
agent_executor = AgentExecutor(
    agent=agent.with_config(configurable={"llm": "claude3"}), tools=tools, verbose=True
)
agent_executor.invoke(
    {"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"},
)



> Entering new AgentExecutor chain...


/Users/chestercurme/repos/langchain/libs/partners/anthropic/langchain_anthropic/chat_models.py:336: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")


82.65606421491815

/Users/chestercurme/repos/langchain/libs/partners/anthropic/langchain_anthropic/chat_models.py:336: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")


85.65606421491815

/Users/chestercurme/repos/langchain/libs/partners/anthropic/langchain_anthropic/chat_models.py:336: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")


-900.8841

/Users/chestercurme/repos/langchain/libs/partners/anthropic/langchain_anthropic/chat_models.py:336: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")




> Finished chain.


{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': 'Therefore, 17.24 - 918.1241 = -900.8841'}

# LangGraph

In [5]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def should_continue(state):
    return "continue" if state["messages"][-1].tool_calls is not None else "end"


def call_model(state, config):
    return {"messages": [llm_with_tools.invoke(state["messages"], config=config)]}


def _invoke_tool(tool_call):
    tool = {tool.name: tool for tool in tools}[tool_call.name]
    return ToolMessage(tool.invoke(tool_call.args), tool_call_id=tool_call.id)


tool_executor = RunnableLambda(_invoke_tool)


def call_tools(state):
    last_message = state["messages"][-1]
    return {"messages": tool_executor.batch(last_message.tool_calls)}


workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")
graph = workflow.compile()

In [6]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"
            )
        ]
    }
)

{'messages': [HumanMessage(content="what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kbBUUeqK75fZZqDTvu8aif7Z', 'function': {'arguments': '{"x": 8, "y": 2.743}', 'name': 'exponentiate'}, 'type': 'function'}, {'id': 'call_pBD8daSyXidXnrIyG4vG5C9O', 'function': {'arguments': '{"x": 17.24, "y": -918.1241}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 168, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8e1d9687-611c-4c8e-9fcd-ef6e48bd22a6-0', tool_calls=[ToolCall(name='exponentiate', args={'x': 8, 'y': 2.743}, id='call_kbBUUeqK75fZZqDTvu8aif7Z'), ToolCall(name='add', args={'x': 17.24, 'y': -918.1241}, id='call_pBD8daSyXidXnrIyG4vG5C9O')]),
  ToolMessage(content='300.03770462067547', tool_call_id='call_kbBUUeqK75fZZqDTvu8aif

In [7]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"
            )
        ]
    },
    config={"configurable": {"llm": "claude3"}},
)

{'messages': [HumanMessage(content="what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"),
  AIMessage(content=[{'text': "Okay, let's break this down into steps:", 'type': 'text'}, {'id': 'toolu_01DJkSDpB8ztmJx2DLNbc3eW', 'input': {'x': 5, 'y': 2.743}, 'name': 'exponentiate', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KuVNohyJr24cPhJkY3XVtt', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 450, 'output_tokens': 84}}, id='run-336cdfb6-0fe4-4d7a-9946-9f01c2eb41ae-0', tool_calls=[ToolCall(name='exponentiate', args={'x': 5, 'y': 2.743}, id='toolu_01DJkSDpB8ztmJx2DLNbc3eW', index=1)]),
  ToolMessage(content='82.65606421491815', tool_call_id='toolu_01DJkSDpB8ztmJx2DLNbc3eW'),
  AIMessage(content=[{'text': 'To get 5 raised to the 2.743 power.', 'type': 'text'}, {'id': 'toolu_01MKQqnDw5CtyuKjQP8YG1FX', 'input': {'x': 3, 'y': 82.65606421491815}, 'name': 'add', 'type': 'tool_use'}], response_metadata={'id':